In [32]:
!pip install --upgrade pip tqdm jupysql pandas numpy matplotlib requests ratelimit ipywidgets python-dotenv

In [33]:
import requests
import pickle
import os
from tqdm import tqdm
import time
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from collections import Counter
from ratelimit import limits, sleep_and_retry
import ipywidgets as widgets
from ipywidgets import interact
from dotenv import load_dotenv

In [27]:
# Load environment variables from .env file
load_dotenv()

# Retrieve the token from the environment variable
token = os.getenv('GITHUB_TOKEN')

# Constants
BASE_DIR = os.path.join(os.path.expanduser('~'), 'dev/stargazers/data/')
DB_FILE = os.path.join(BASE_DIR, 'stargazers.db')
UPDATED_DB_FILE = os.path.join(BASE_DIR, 'updated_stargazers.db')
PKL_FILE = os.path.join(BASE_DIR, 'stargazers.pkl')

if not os.path.exists(BASE_DIR):
    os.makedirs(BASE_DIR)

In [ ]:
!pip install --upgrade pip tqdm jupysql pandas numpy matplotlib requests ratelimit ipywidgets python-dotenv

In [ ]:
import requests
import pickle
import os
from tqdm import tqdm
import time
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from collections import Counter
from ratelimit import limits, sleep_and_retry
import ipywidgets as widgets
from ipywidgets import interact
from dotenv import load_dotenv

In [ ]:
# Load environment variables from .env file
load_dotenv()

# Retrieve the token from the environment variable
token = os.getenv('GITHUB_TOKEN')

# Constants
BASE_DIR = os.path.join(os.path.expanduser('~'), 'dev/stargazers/data/')
DB_FILE = os.path.join(BASE_DIR, 'stargazers.db')
UPDATED_DB_FILE = os.path.join(BASE_DIR, 'updated_stargazers.db')
PKL_FILE = os.path.join(BASE_DIR, 'stargazers.pkl')

if not os.path.exists(BASE_DIR):
    os.makedirs(BASE_DIR)

In [ ]:
# Declare global variables
repo = 'daytona'
org = 'daytonaio'

# Function to handle input and display results
def get_repo_details(new_org='daytonaio', new_repo='daytona'):
    global org, repo
    org = new_org
    repo = new_repo
    print(f'Owner: {org}')
    print(f'Repo: {repo}')

# Create interactive widgets
interact(get_repo_details, new_org=org, new_repo=repo)

interactive(children=(Text(value='daytonaio', description='new_org'), Text(value='daytona', description='new_r…

<function __main__.get_repo_details(new_org='daytonaio', new_repo='daytona')>

In [ ]:
# Block 4: Fetch stargazers in batches and save to pkl file
def fetch_stargazers(org, repo, last_record_id):
    stargazers_url = f'https://api.github.com/repos/{org}/{repo}/stargazers'
    headers = {
        'Accept': 'application/vnd.github.v3.star+json',
        'Authorization': f'token {token}'
    }
    stargazers = []
    page = 1
    per_page = 500

    while True:
        response = requests.get(stargazers_url, headers=headers, params={'page': page, 'per_page': per_page, 'since': last_record_id})

        if response.status_code != 200:
            print(f"Failed to fetch data: {response.status_code} - {response.text}")
            break

        data = response.json()
        if not data:
            break

        stargazers.extend(data)
        page += 1

    return stargazers

def save_stargazers_to_pkl(stargazers, filename):
    with open(filename, 'wb') as file:
        pickle.dump(stargazers, file)

    print(f"Stargazers data saved to {filename}")

def load_stargazers_from_pkl(filename):
    if os.path.exists(filename):
        with open(filename, 'rb') as file:
            stargazers = pickle.load(file)
        return stargazers
    else:
        print(f"File {filename} not found.")
        return None

In [ ]:
# Block 6: Populate SQLite database using jupysql
def populate_database(stargazers):
    conn = sqlite3.connect(DB_FILE)
    c = conn.cursor()

    c.execute('''CREATE TABLE IF NOT EXISTS stargazers
                 (id INTEGER PRIMARY KEY,
                  username TEXT,
                  starred_at TEXT)''')

    for stargazer in stargazers:
        c.execute("INSERT INTO stargazers (username, starred_at) VALUES (?, ?)",
                  (stargazer['user']['login'], stargazer['starred_at']))

    conn.commit()
    conn.close()

In [ ]:
# Block 7: Fetch stargazer user data and update SQLite database
@sleep_and_retry
@limits(calls=4500, period=3600)  # Adjust the rate limit as per your requirements
def fetch_user_data(stargazers):
    headers = {
        'Accept': 'application/vnd.github.v3.star+json',
        'Authorization': f'token {token}'
    }

    updated_stargazers = []

    for stargazer in tqdm(stargazers, desc="Fetching user data"):
        user_url = stargazer['user']['url']

        try:
            response = requests.get(user_url, headers=headers)
            response.raise_for_status()  # Raise an exception for 4xx or 5xx status codes

            user_data = response.json()
            stargazer['user']['profile_data'] = user_data
            updated_stargazers.append(stargazer)

        except requests.exceptions.RequestException as e:
            print(f"Failed to fetch data for {user_url}: {str(e)}")

        except Exception as e:
            print(f"An error occurred while fetching data for {user_url}: {str(e)}")

    return updated_stargazers

def update_database(stargazers):
    conn = sqlite3.connect(DB_FILE)
    c = conn.cursor()

    c.execute('''CREATE TABLE IF NOT EXISTS users
                 (id INTEGER PRIMARY KEY,
                  login TEXT,
                  avatar_url TEXT,
                  html_url TEXT,
                  followers_url TEXT,
                  following_url TEXT,
                  gists_url TEXT,
                  starred_url TEXT,
                  subscriptions_url TEXT,
                  organizations_url TEXT,
                  repos_url TEXT,
                  events_url TEXT,
                  received_events_url TEXT,
                  type TEXT,
                  site_admin INTEGER,
                  name TEXT,
                  company TEXT,
                  blog TEXT,
                  location TEXT,
                  email TEXT,
                  hireable INTEGER,
                  bio TEXT,
                  twitter_username TEXT,
                  public_repos INTEGER,
                  public_gists INTEGER,
                  followers INTEGER,
                  following INTEGER,
                  created_at TEXT,
                  updated_at TEXT)''')

    for stargazer in stargazers:
        user_data = stargazer['user']['profile_data']

        fields = (
            user_data['id'],
            user_data['login'],
            user_data.get('avatar_url', ''),
            user_data.get('html_url', ''),
            user_data.get('followers_url', ''),
            user_data.get('following_url', ''),
            user_data.get('gists_url', ''),
            user_data.get('starred_url', ''),
            user_data.get('subscriptions_url', ''),
            user_data.get('organizations_url', ''),
            user_data.get('repos_url', ''),
            user_data.get('events_url', ''),
            user_data.get('received_events_url', ''),
            user_data.get('type', ''),
            int(user_data.get('site_admin', False)),
            user_data.get('name', ''),
            user_data.get('company', ''),
            user_data.get('blog', ''),
            user_data.get('location', ''),
            user_data.get('email', ''),
            int(user_data.get('hireable', False)) if user_data.get('hireable') is not None else None,
            user_data.get('bio', ''),
            user_data.get('twitter_username', ''),
            user_data.get('public_repos', 0),
            user_data.get('public_gists', 0),
            user_data.get('followers', 0),
            user_data.get('following', 0),
            user_data.get('created_at', ''),
            user_data.get('updated_at', '')
        )

        c.execute('''INSERT OR REPLACE INTO users
                     (id, login, avatar_url, html_url, followers_url, following_url, gists_url, starred_url,
                      subscriptions_url, organizations_url, repos_url, events_url, received_events_url, type,
                      site_admin, name, company, blog, location, email, hireable, bio, twitter_username,
                      public_repos, public_gists, followers, following, created_at, updated_at)
                     VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)''', fields)

    conn.commit()
    conn.close()

In [ ]:
# %%
# %%
def load_stargazers_from_db():
    conn = sqlite3.connect(DB_FILE)
    c = conn.cursor()

    c.execute("SELECT id, username, starred_at FROM stargazers")
    rows = c.fetchall()

    # Convert to a list of dictionaries to match the format
    stargazers = [
        {
            'id': row[0],
            'user': {
                'login': row[1],
                'url': f'https://api.github.com/users/{row[1]}'  # Construct the URL for user details
            },
            'starred_at': row[2]
        }
        for row in rows
    ]

    conn.close()
    return stargazers

def get_last_fetched_user_id():
    conn = sqlite3.connect(DB_FILE)
    c = conn.cursor()

    c.execute("SELECT id FROM users ORDER BY id DESC LIMIT 1")
    last_user = c.fetchone()

    conn.close()
    return last_user[0] if last_user else None

# %%
def main():
    # Load stargazers from the database
    stargazers = load_stargazers_from_db()

    # Get the ID of the last fetched user
    last_user_id = get_last_fetched_user_id()
    if last_user_id:
        print(f"Last fetched user ID: {last_user_id}")
        remaining_stargazers = [stargazer for stargazer in stargazers if stargazer['id'] > last_user_id]
    else:
        print("No users in database. Fetching details for all stargazers.")
        remaining_stargazers = stargazers

    # Fetch user data for the remaining stargazers
    if remaining_stargazers:
        updated_stargazers = fetch_user_data(remaining_stargazers)

        # Update the database with the new user data
        update_database(updated_stargazers)
    else:
        print("All stargazers are up to date.")

# %%
# Call the main function
main()

Last fetched user ID: 4809


Fetching user data: 100%|██████████| 1301/1301 [06:27<00:00,  3.36it/s]


In [ ]:
import sqlite3
import os

# Define file paths
BASE_DIR = os.path.join(os.path.expanduser('~'), 'dev/stargazers/data/')
DB_FILE = os.path.join(BASE_DIR, 'stargazers.db')
UPDATED_DB_FILE = os.path.join(BASE_DIR, 'updated_stargazers.db')
PKL_FILE = os.path.join(BASE_DIR, 'stargazers.pkl')

# Connect to the source and destination databases
conn_source = sqlite3.connect(UPDATED_DB_FILE)
conn_dest = sqlite3.connect(DB_FILE)

cursor_source = conn_source.cursor()
cursor_dest = conn_dest.cursor()

# Find the current maximum id in the destination database
cursor_dest.execute("SELECT MAX(id) FROM users")
max_id = cursor_dest.fetchone()[0]
if max_id is None:
    max_id = 0

# Extract users from the source database
cursor_source.execute("SELECT * FROM users")
users = cursor_source.fetchall()

# Assuming we have 5 columns, and id is the first one
# Insert users into the destination database, incrementing the id
for user in users:
    # Generate a new id by incrementing the max_id
    max_id += 1
    # Create a new record with the new id
    new_user = (max_id,) + user[1:]  # Ignore the id from the source
    cursor_dest.execute("INSERT INTO users VALUES (?, ?, ?, ?, ?)", new_user)  # Adjust based on the number of columns

# Commit the changes and close the connections
conn_dest.commit()
conn_source.close()
conn_dest.close()

print("Users have been successfully copied from updated_stargazers.db to stargazers.db with incremented id")

Users have been successfully copied from updated_stargazers.db to stargazers.db with incremented id


In [19]:
# Declare global variables
repo = 'daytona'
org = 'daytonaio'

# Function to handle input and display results
def get_repo_details(new_org='daytonaio', new_repo='daytona'):
    global org, repo
    org = new_org
    repo = new_repo
    print(f'Owner: {org}')
    print(f'Repo: {repo}')

# Create interactive widgets
interact(get_repo_details, new_org=org, new_repo=repo)

interactive(children=(Text(value='daytonaio', description='new_org'), Text(value='daytona', description='new_r…

<function __main__.get_repo_details(new_org='daytonaio', new_repo='daytona')>

In [20]:
# Block 4: Fetch stargazers in batches and save to pkl file
def fetch_stargazers(org, repo, last_record_id):
    stargazers_url = f'https://api.github.com/repos/{org}/{repo}/stargazers'
    headers = {
        'Accept': 'application/vnd.github.v3.star+json',
        'Authorization': f'token {token}'
    }
    stargazers = []
    page = 1
    per_page = 500

    while True:
        response = requests.get(stargazers_url, headers=headers, params={'page': page, 'per_page': per_page, 'since': last_record_id})

        if response.status_code != 200:
            print(f"Failed to fetch data: {response.status_code} - {response.text}")
            break

        data = response.json()
        if not data:
            break

        stargazers.extend(data)
        page += 1

    return stargazers

def save_stargazers_to_pkl(stargazers, filename):
    with open(filename, 'wb') as file:
        pickle.dump(stargazers, file)

    print(f"Stargazers data saved to {filename}")

def load_stargazers_from_pkl(filename):
    if os.path.exists(filename):
        with open(filename, 'rb') as file:
            stargazers = pickle.load(file)
        return stargazers
    else:
        print(f"File {filename} not found.")
        return None

In [21]:
# Block 6: Populate SQLite database using jupysql
def populate_database(stargazers):
    conn = sqlite3.connect(DB_FILE)
    c = conn.cursor()

    c.execute('''CREATE TABLE IF NOT EXISTS stargazers
                 (id INTEGER PRIMARY KEY,
                  username TEXT,
                  starred_at TEXT)''')

    for stargazer in stargazers:
        c.execute("INSERT INTO stargazers (username, starred_at) VALUES (?, ?)",
                  (stargazer['user']['login'], stargazer['starred_at']))

    conn.commit()
    conn.close()

In [22]:
# Block 7: Fetch stargazer user data and update SQLite database
@sleep_and_retry
@limits(calls=4500, period=3600)  # Adjust the rate limit as per your requirements
def fetch_user_data(stargazers):
    headers = {
        'Accept': 'application/vnd.github.v3.star+json',
        'Authorization': f'token {token}'
    }

    updated_stargazers = []

    for stargazer in tqdm(stargazers, desc="Fetching user data"):
        user_url = stargazer['user']['url']

        try:
            response = requests.get(user_url, headers=headers)
            response.raise_for_status()  # Raise an exception for 4xx or 5xx status codes

            user_data = response.json()
            stargazer['user']['profile_data'] = user_data
            updated_stargazers.append(stargazer)

        except requests.exceptions.RequestException as e:
            print(f"Failed to fetch data for {user_url}: {str(e)}")

        except Exception as e:
            print(f"An error occurred while fetching data for {user_url}: {str(e)}")

    return updated_stargazers

def update_database(stargazers):
    conn = sqlite3.connect(DB_FILE)
    c = conn.cursor()

    c.execute('''CREATE TABLE IF NOT EXISTS users
                 (id INTEGER PRIMARY KEY,
                  login TEXT,
                  avatar_url TEXT,
                  html_url TEXT,
                  followers_url TEXT,
                  following_url TEXT,
                  gists_url TEXT,
                  starred_url TEXT,
                  subscriptions_url TEXT,
                  organizations_url TEXT,
                  repos_url TEXT,
                  events_url TEXT,
                  received_events_url TEXT,
                  type TEXT,
                  site_admin INTEGER,
                  name TEXT,
                  company TEXT,
                  blog TEXT,
                  location TEXT,
                  email TEXT,
                  hireable INTEGER,
                  bio TEXT,
                  twitter_username TEXT,
                  public_repos INTEGER,
                  public_gists INTEGER,
                  followers INTEGER,
                  following INTEGER,
                  created_at TEXT,
                  updated_at TEXT)''')

    for stargazer in stargazers:
        user_data = stargazer['user']['profile_data']

        fields = (
            user_data['id'],
            user_data['login'],
            user_data.get('avatar_url', ''),
            user_data.get('html_url', ''),
            user_data.get('followers_url', ''),
            user_data.get('following_url', ''),
            user_data.get('gists_url', ''),
            user_data.get('starred_url', ''),
            user_data.get('subscriptions_url', ''),
            user_data.get('organizations_url', ''),
            user_data.get('repos_url', ''),
            user_data.get('events_url', ''),
            user_data.get('received_events_url', ''),
            user_data.get('type', ''),
            int(user_data.get('site_admin', False)),
            user_data.get('name', ''),
            user_data.get('company', ''),
            user_data.get('blog', ''),
            user_data.get('location', ''),
            user_data.get('email', ''),
            int(user_data.get('hireable', False)) if user_data.get('hireable') is not None else None,
            user_data.get('bio', ''),
            user_data.get('twitter_username', ''),
            user_data.get('public_repos', 0),
            user_data.get('public_gists', 0),
            user_data.get('followers', 0),
            user_data.get('following', 0),
            user_data.get('created_at', ''),
            user_data.get('updated_at', '')
        )

        c.execute('''INSERT OR REPLACE INTO users
                     (id, login, avatar_url, html_url, followers_url, following_url, gists_url, starred_url,
                      subscriptions_url, organizations_url, repos_url, events_url, received_events_url, type,
                      site_admin, name, company, blog, location, email, hireable, bio, twitter_username,
                      public_repos, public_gists, followers, following, created_at, updated_at)
                     VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)''', fields)

    conn.commit()
    conn.close()

In [25]:
# %%
# %%
def load_stargazers_from_db():
    conn = sqlite3.connect(DB_FILE)
    c = conn.cursor()

    c.execute("SELECT id, username, starred_at FROM stargazers")
    rows = c.fetchall()

    # Convert to a list of dictionaries to match the format
    stargazers = [
        {
            'id': row[0],
            'user': {
                'login': row[1],
                'url': f'https://api.github.com/users/{row[1]}'  # Construct the URL for user details
            },
            'starred_at': row[2]
        }
        for row in rows
    ]

    conn.close()
    return stargazers

def get_last_fetched_user_id():
    conn = sqlite3.connect(DB_FILE)
    c = conn.cursor()

    c.execute("SELECT id FROM users ORDER BY id DESC LIMIT 1")
    last_user = c.fetchone()

    conn.close()
    return last_user[0] if last_user else None

# %%
def main():
    # Load stargazers from the database
    stargazers = load_stargazers_from_db()

    # Get the ID of the last fetched user
    last_user_id = get_last_fetched_user_id()
    if last_user_id:
        print(f"Last fetched user ID: {last_user_id}")
        remaining_stargazers = [stargazer for stargazer in stargazers if stargazer['id'] > last_user_id]
    else:
        print("No users in database. Fetching details for all stargazers.")
        remaining_stargazers = stargazers

    # Fetch user data for the remaining stargazers
    if remaining_stargazers:
        updated_stargazers = fetch_user_data(remaining_stargazers)

        # Update the database with the new user data
        update_database(updated_stargazers)
    else:
        print("All stargazers are up to date.")

# %%
# Call the main function
main()

Last fetched user ID: 4809


Fetching user data: 100%|██████████| 1301/1301 [06:27<00:00,  3.36it/s]
